In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
# do pip install py-openaq in the pandas environment
import openaq
from pprint import pprint


In [2]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [9]:
engine = create_engine("sqlite:///output_data/airquality2.sqlite")

In [5]:
# api = openaq.OpenAQ()
# res = api.measurements(city='St. Louis', parameter='pm25', limit=10000, df=True)
# res = res.query("value >= 0.0")
# # g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)
# # g.map(plt.hist, "value")
# # print(res)
# res.dtypes
# stl_df = pd.DataFrame(res)
# stl_df

api = openaq.OpenAQ()
res = api.measurements(country='US', parameter='pm25', limit=10000, df=True)

res = res.query("value >= 0.0")
# g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# g.map(plt.hist, "value")
# print(res)
res.dtypes

us_df_pm25 = pd.DataFrame(res)
us_df_pm25.head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2019-11-16 10:00:00,Blue Ridge I-70,pm25,7.4,b'\xc2\xb5g/m\xc2\xb3',US,Kansas City,2019-11-16 16:00:00+00:00,39.047911,-94.450513
2019-11-16 10:00:00,ARNOLD WEST,pm25,18.4,b'\xc2\xb5g/m\xc2\xb3',US,St. Louis,2019-11-16 16:00:00+00:00,38.448581,-90.398436
2019-11-16 10:00:00,Ladue,pm25,23.1,b'\xc2\xb5g/m\xc2\xb3',US,St. Louis,2019-11-16 16:00:00+00:00,38.650259,-90.350463
2019-11-16 10:00:00,BROADWAY (South),pm25,17.1,b'\xc2\xb5g/m\xc2\xb3',US,St. Louis,2019-11-16 16:00:00+00:00,38.542500,-90.263611
2019-11-16 10:00:00,Branch Street,pm25,19.7,b'\xc2\xb5g/m\xc2\xb3',US,ST. LOUIS (CITY),2019-11-16 16:00:00+00:00,38.656430,-90.189770


In [23]:
pm25_groupby = us_df_pm25.groupby('location')
pm25_first = pm25_groupby.first()

pm25mean = us_df_pm25.groupby('location').mean()
pm25mean = pm25mean.rename(columns = {"value": "avgvalue", "coordinates.latitude": "lat", "coordinates.longitude": "lng"})

pm25 = pd.merge(pm25_first, pm25mean, on = 'location')
pm25 = pm25.drop(['value','lat','lng'], axis=1)
pm25 = pm25.rename(columns = {"coordinates.latitude": "coordinates_latitude", "coordinates.longitude": "coordinates_longitude"})

# create table schema ourselves
engine.execute("""CREATE TABLE pm25 (
        location TEXT,
        parameter TEXT,
        unit TEXT,
        country TEXT,
        city TEXT,
        "date.utc" TIMESTAMP,
        "coordinates_latitude" FLOAT,
        "coordinates_longitude" FLOAT,
        avgvalue FLOAT
, primary key("location"));""")

pm25.to_sql(name='pm25', con=engine, index=True, index_label='location',if_exists='append')

pm25.head()

,parameter,unit,country,city,date.utc,coordinates_latitude,coordinates_longitude,avgvalue
location,,,,,,,,
2912 Coffey,pm25,b'\xc2\xb5g/m\xc2\xb3',US,Omaha-Council Bluffs,2019-11-16 16:00:00+00:00,41.133317,-95.956078,8.500000
ADEQ-DSR,pm25,b'\xc2\xb5g/m\xc2\xb3',US,Little Rock-North Little Rock,2019-11-16 10:00:00+00:00,34.681100,-92.328300,13.450000
AIR LAB,pm25,b'\xc2\xb5g/m\xc2\xb3',US,Knoxville,2019-11-16 16:00:00+00:00,35.980300,-83.930600,8.178571
ALLEN PARK,pm25,b'\xc2\xb5g/m\xc2\xb3',US,Detroit-Warren-Livonia,2019-11-16 16:00:00+00:00,42.228300,-83.209200,5.178571
APOLLO BEACH,pm25,b'\xc2\xb5g/m\xc2\xb3',US,HILLSBOROUGH,2019-11-16 16:00:00+00:00,27.779712,-82.419835,2.600000


In [7]:
api = openaq.OpenAQ()
res = api.measurements(country='US', parameter='bc', limit=9999, df=True)

res = res.query("value >= 0.0")
# g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# g.map(plt.hist, "value")
# print(res)
res.dtypes

us_df_bc = pd.DataFrame(res)
us_df_bc.head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2019-11-16 08:00:00,Berkeley Aquatic Par,bc,0.51,b'\xc2\xb5g/m\xc2\xb3',US,ALAMEDA,2019-11-16 16:00:00+00:00,37.864767,-122.302741
2019-11-16 08:00:00,Arden Arcade - Del P,bc,0.00,b'\xc2\xb5g/m\xc2\xb3',US,Sacramento--Arden-Arcade--Roseville,2019-11-16 16:00:00+00:00,38.613804,-121.368007
2019-11-16 08:00:00,Portland Near Road,bc,1.80,b'\xc2\xb5g/m\xc2\xb3',US,Portland-Vancouver-Beaverton,2019-11-16 16:00:00+00:00,45.399160,-122.745500
2019-11-16 10:00:00,Gary-IITRI,bc,0.40,b'\xc2\xb5g/m\xc2\xb3',US,Chicago-Naperville-Joliet,2019-11-16 16:00:00+00:00,41.606700,-87.304700
2019-11-16 11:00:00,Indpls-Washington Pa,bc,1.33,b'\xc2\xb5g/m\xc2\xb3',US,Indianapolis,2019-11-16 16:00:00+00:00,39.810800,-86.114700


In [24]:
bc_groupby = us_df_bc.groupby('location')
bc_first = bc_groupby.first()

bcmean = us_df_bc.groupby('location').mean()
bcmean = bcmean.rename(columns = {"value": "avgvalue", "coordinates.latitude": "lat", "coordinates.longitude": "lng"})

bc = pd.merge(bc_first, bcmean, on = 'location')
bc = bc.drop(['value','lat','lng'], axis=1)
bc = bc.rename(columns = {"coordinates.latitude": "coordinates_latitude", "coordinates.longitude": "coordinates_longitude"})

engine.execute("""CREATE TABLE bc (
        location TEXT,
        parameter TEXT,
        unit TEXT,
        country TEXT,
        city TEXT,
        "date.utc" TIMESTAMP,
        "coordinates_latitude" FLOAT,
        "coordinates_longitude" FLOAT,
        avgvalue FLOAT
, primary key("location"));""")

bc.to_sql(name='bc', con=engine, index=True, index_label='location', if_exists='append')

bc.head()

,parameter,unit,country,city,date.utc,coordinates_latitude,coordinates_longitude,avgvalue
location,,,,,,,,
Arden Arcade - Del P,bc,b'\xc2\xb5g/m\xc2\xb3',US,Sacramento--Arden-Arcade--Roseville,2019-11-16 16:00:00+00:00,38.613804,-121.368007,0.827586
Berkeley Aquatic Par,bc,b'\xc2\xb5g/m\xc2\xb3',US,ALAMEDA,2019-11-16 16:00:00+00:00,37.864767,-122.302741,1.256881
E Providence,bc,b'\xc2\xb5g/m\xc2\xb3',US,Providence-New Bedford-Fall River,2019-11-16 13:00:00+00:00,41.840302,-71.361702,0.397548
Elkhart-Prairie St.,bc,b'\xc2\xb5g/m\xc2\xb3',US,Elkhart-Goshen,2019-11-16 16:00:00+00:00,41.656944,-85.968330,4.012271
Evansville-Buena Vis,bc,b'\xc2\xb5g/m\xc2\xb3',US,Evansville,2019-11-16 16:00:00+00:00,38.013300,-87.577200,0.589354


In [12]:
api = openaq.OpenAQ()
res = api.measurements(country='US', parameter='pm10', limit=10000, df=True)

res = res.query("value >= 0.0")
# g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# g.map(plt.hist, "value")
# print(res)
res.dtypes

us_df_pm10 = pd.DataFrame(res)
us_df_pm10.head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2019-11-16 11:00:00,CARTHAGE,pm10,8,b'\xc2\xb5g/m\xc2\xb3',US,Joplin,2019-11-16 17:00:00+00:00,37.198220,-94.317020
2019-11-16 11:00:00,ARNOLD WEST,pm10,12,b'\xc2\xb5g/m\xc2\xb3',US,St. Louis,2019-11-16 17:00:00+00:00,38.448581,-90.398436
2019-11-16 11:00:00,MTSP,pm10,14,b'\xc2\xb5g/m\xc2\xb3',US,MONROE,2019-11-16 17:00:00+00:00,39.475100,-91.788990
2019-11-16 11:00:00,Branch Street,pm10,20,b'\xc2\xb5g/m\xc2\xb3',US,ST. LOUIS (CITY),2019-11-16 17:00:00+00:00,38.656430,-90.189770
2019-11-16 09:00:00,Mesa 2 /Nipomo,pm10,43,b'\xc2\xb5g/m\xc2\xb3',US,San Luis Obispo-Paso Robles,2019-11-16 17:00:00+00:00,35.020790,-120.563890


In [25]:
pm10_groupby = us_df_pm10.groupby('location')
pm10_first = pm10_groupby.first()

pm10mean = us_df_pm10.groupby('location').mean()
pm10mean = pm10mean.rename(columns = {"value": "avgvalue", "coordinates.latitude": "lat", "coordinates.longitude": "lng"})

pm10 = pd.merge(pm10_first, pm10mean, on = 'location')
pm10 = pm10.drop(['value','lat','lng'], axis=1)
pm10 = pm10.rename(columns = {"coordinates.latitude": "coordinates_latitude", "coordinates.longitude": "coordinates_longitude"})

engine.execute("""CREATE TABLE pm10 (
        location TEXT,
        parameter TEXT,
        unit TEXT,
        country TEXT,
        city TEXT,
        "date.utc" TIMESTAMP,
        "coordinates_latitude" FLOAT,
        "coordinates_longitude" FLOAT,
        avgvalue FLOAT
, primary key("location"));""")

pm10.to_sql(name='pm10', con=engine, index=True, index_label='location',if_exists='append')

pm10.head()

,parameter,unit,country,city,date.utc,coordinates_latitude,coordinates_longitude,avgvalue
location,,,,,,,,
7E Deming Airport,pm10,b'\xc2\xb5g/m\xc2\xb3',US,Deming,2019-11-16 13:00:00+00:00,32.255800,-107.722800,21.457143
ARNOLD,pm10,b'\xc2\xb5g/m\xc2\xb3',US,St. Louis,2019-11-15 13:00:00+00:00,38.448581,-90.398436,14.222222
ARNOLD WEST,pm10,b'\xc2\xb5g/m\xc2\xb3',US,St. Louis,2019-11-16 17:00:00+00:00,38.448581,-90.398436,19.296296
Ajo,pm10,b'\xc2\xb5g/m\xc2\xb3',US,PIMA,2019-11-16 16:00:00+00:00,32.382034,-112.857452,32.571429
Alamo Lake,pm10,b'\xc2\xb5g/m\xc2\xb3',US,LA PAZ,2019-11-16 16:00:00+00:00,34.243889,-113.558611,15.763158


In [14]:
api = openaq.OpenAQ()
res = api.measurements(country='US', parameter='so2', limit=10000, df=True)

res = res.query("value >= 0.0")
# g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# g.map(plt.hist, "value")
# print(res)
res.dtypes

us_df_so2 = pd.DataFrame(res)
us_df_so2.head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2019-11-16 11:00:00,"Herculaneum, Mott St",so2,0.001,ppm,US,St. Louis,2019-11-16 17:00:00+00:00,38.263394,-90.379667
2019-11-16 11:00:00,Rider Trail I-70,so2,0.002,ppm,US,ST. LOUIS,2019-11-16 17:00:00+00:00,38.752640,-90.448840
2019-11-16 11:00:00,BLAIR STREET,so2,0.001,ppm,US,St. Louis,2019-11-16 17:00:00+00:00,38.656449,-90.198548
2019-11-16 11:00:00,Sheldon Station,so2,0.000,ppm,US,LANCASTER,2019-11-16 17:00:00+00:00,40.554760,-96.780350
2019-11-16 09:00:00,Mesa 2 /Nipomo,so2,0.000,ppm,US,San Luis Obispo-Paso Robles,2019-11-16 17:00:00+00:00,35.020790,-120.563890


In [26]:
so2_groupby = us_df_so2.groupby('location')
so2_first = so2_groupby.first()

so2mean = us_df_so2.groupby('location').mean()
so2mean = so2mean.rename(columns = {"value": "avgvalue", "coordinates.latitude": "lat", "coordinates.longitude": "lng"})

so2 = pd.merge(so2_first, so2mean, on = 'location')
so2 = so2.drop(['value','lat','lng'], axis=1)
so2 = so2.rename(columns = {"coordinates.latitude": "coordinates_latitude", "coordinates.longitude": "coordinates_longitude"})

engine.execute("""CREATE TABLE so2 (
        location TEXT,
        parameter TEXT,
        unit TEXT,
        country TEXT,
        city TEXT,
        "date.utc" TIMESTAMP,
        "coordinates_latitude" FLOAT,
        "coordinates_longitude" FLOAT,
        avgvalue FLOAT
, primary key("location"));""")

so2.to_sql(name='so2', con=engine, index=True, index_label='location',if_exists='append')

so2.head()

,parameter,unit,country,city,date.utc,coordinates_latitude,coordinates_longitude,avgvalue
location,,,,,,,,
16th and Whitmore,so2,ppm,US,Omaha-Council Bluffs,2019-11-16 16:00:00+00:00,41.322472,-95.937992,0.000211
ALLEN PARK,so2,ppm,US,Detroit-Warren-Livonia,2019-11-16 16:00:00+00:00,42.228300,-83.209200,0.001196
APOLLO BEACH,so2,ppm,US,HILLSBOROUGH,2019-11-16 16:00:00+00:00,27.779712,-82.419835,0.000326
Acadia NP - McFarlan,so2,ppm,US,HANCOCK,2019-11-16 16:00:00+00:00,44.377048,-68.260902,0.000000
Akron NIFH/STEM,so2,ppm,US,Akron,2019-11-16 16:00:00+00:00,41.079140,-81.516270,0.000000


In [16]:
api = openaq.OpenAQ()
res = api.measurements(country='US', parameter='no2', limit=10000, df=True)

res = res.query("value >= 0.0")
# g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# g.map(plt.hist, "value")
# print(res)
res.dtypes

us_df_no2 = pd.DataFrame(res)
us_df_no2.head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2019-11-16 11:00:00,MTSP,no2,0.005,ppm,US,MONROE,2019-11-16 17:00:00+00:00,39.475100,-91.788990
2019-11-16 11:00:00,Rider Trail I-70,no2,0.008,ppm,US,ST. LOUIS,2019-11-16 17:00:00+00:00,38.752640,-90.448840
2019-11-16 11:00:00,BLAIR STREET,no2,0.008,ppm,US,St. Louis,2019-11-16 17:00:00+00:00,38.656449,-90.198548
2019-11-16 11:00:00,Forest Park,no2,0.016,ppm,US,St. Louis,2019-11-16 17:00:00+00:00,38.631057,-90.281144
2019-11-16 09:00:00,Nipomo RP,no2,0.002,ppm,US,San Luis Obispo-Paso Robles,2019-11-16 17:00:00+00:00,35.031500,-120.501010


In [27]:
no2_groupby = us_df_no2.groupby('location')
no2_first = no2_groupby.first()

no2mean = us_df_no2.groupby('location').mean()
no2mean = no2mean.rename(columns = {"value": "avgvalue", "coordinates.latitude": "lat", "coordinates.longitude": "lng"})

no2 = pd.merge(no2_first, no2mean, on = 'location')
no2 = no2.drop(['value','lat','lng'], axis=1)
no2 = no2.rename(columns = {"coordinates.latitude": "coordinates_latitude", "coordinates.longitude": "coordinates_longitude"})

engine.execute("""CREATE TABLE no2 (
        location TEXT,
        parameter TEXT,
        unit TEXT,
        country TEXT,
        city TEXT,
        "date.utc" TIMESTAMP,
        "coordinates_latitude" FLOAT,
        "coordinates_longitude" FLOAT,
        avgvalue FLOAT
, primary key("location"));""")

no2.to_sql(name='no2', con=engine, index=True, index_label='location',if_exists='append')

no2.head()

,parameter,unit,country,city,date.utc,coordinates_latitude,coordinates_longitude,avgvalue
location,,,,,,,,
60 Near Road,no2,ppm,US,SAN BERNARDINO,2019-11-16 16:00:00+00:00,34.030900,-117.617403,0.035690
710 Near Road,no2,ppm,US,LOS ANGELES,2019-11-16 16:00:00+00:00,33.859662,-118.200707,0.033214
AURORA HILLS,no2,ppm,US,Washington-Arlington-Alexandria,2019-11-16 16:00:00+00:00,38.859402,-77.058899,0.023024
Anaheim,no2,ppm,US,Los Angeles-Long Beach-Santa Ana,2019-11-16 16:00:00+00:00,33.830586,-117.938509,0.020738
Anaheim Near Road,no2,ppm,US,ORANGE,2019-11-16 16:00:00+00:00,33.819305,-117.918759,0.022756


In [18]:
api = openaq.OpenAQ()
res = api.measurements(country='US', parameter='o3', limit=10000, df=True)

res = res.query("value >= 0.0")
# g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# g.map(plt.hist, "value")
# print(res)
res.dtypes

us_df_o3 = pd.DataFrame(res)
us_df_o3.head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2019-11-16 11:00:00,MTSP,o3,0.029,ppm,US,MONROE,2019-11-16 17:00:00+00:00,39.475100,-91.788990
2019-11-16 11:00:00,BLAIR STREET,o3,0.021,ppm,US,St. Louis,2019-11-16 17:00:00+00:00,38.656449,-90.198548
2019-11-16 12:00:00,Kittanning,o3,0.038,ppm,US,Pittsburgh,2019-11-16 17:00:00+00:00,40.814200,-79.565000
2019-11-16 12:00:00,Hookstown,o3,0.035,ppm,US,Pittsburgh,2019-11-16 17:00:00+00:00,40.562500,-80.504200
2019-11-16 12:00:00,Brighton Township,o3,0.038,ppm,US,Pittsburgh,2019-11-16 17:00:00+00:00,40.684700,-80.359700


In [28]:
o3_groupby = us_df_o3.groupby('location')
o3_first = o3_groupby.first()

o3mean = us_df_o3.groupby('location').mean()
o3mean = o3mean.rename(columns = {"value": "avgvalue", "coordinates.latitude": "lat", "coordinates.longitude": "lng"})

o3 = pd.merge(o3_first, o3mean, on = 'location')
o3 = o3.drop(['value','lat','lng'], axis=1)
o3 = o3.rename(columns = {"coordinates.latitude": "coordinates_latitude", "coordinates.longitude": "coordinates_longitude"})

# create table schema ourselves
engine.execute("""CREATE TABLE o3 (
        location TEXT,
        parameter TEXT,
        unit TEXT,
        country TEXT,
        city TEXT,
        "date.utc" TIMESTAMP,
        "coordinates_latitude" FLOAT,
        "coordinates_longitude" FLOAT,
        avgvalue FLOAT
, primary key("location"));""")

o3.to_sql(name='o3', con=engine, index=True, index_label="location",if_exists='append')

o3.head()

,parameter,unit,country,city,date.utc,coordinates_latitude,coordinates_longitude,avgvalue
location,,,,,,,,
22nd Street & Craycr,o3,ppm,US,Tucson,2019-11-16 16:00:00+00:00,32.205000,-110.877200,0.027467
2LL Los Lunas,o3,ppm,US,Albuquerque,2019-11-16 13:00:00+00:00,34.814700,-106.739700,0.009455
AJ Maintenance Yard,o3,ppm,US,Phoenix-Mesa-Scottsdale,2019-11-16 16:00:00+00:00,33.420601,-111.503304,0.043786
ALLEN PARK,o3,ppm,US,Detroit-Warren-Livonia,2019-11-16 16:00:00+00:00,42.228300,-83.209200,0.014600
ALSIP,o3,ppm,US,Chicago-Naperville-Joliet,2019-11-16 16:00:00+00:00,41.670800,-87.732500,0.003000


In [20]:
api = openaq.OpenAQ()
res = api.measurements(country='US', parameter='co', limit=10000, df=True)

res = res.query("value >= 0.0")
# g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# g.map(plt.hist, "value")
# print(res)
res.dtypes

us_df_co = pd.DataFrame(res)
us_df_co.head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2019-11-16 11:00:00,BLAIR STREET,co,0.08,ppm,US,St. Louis,2019-11-16 17:00:00+00:00,38.656449,-90.198548
2019-11-16 11:00:00,Forest Park,co,0.32,ppm,US,St. Louis,2019-11-16 17:00:00+00:00,38.631057,-90.281144
2019-11-16 10:00:00,Ute 1,co,0.20,ppm,US,Durango,2019-11-16 17:00:00+00:00,37.136780,-107.628630
2019-11-16 12:00:00,NCORE,co,0.13,ppm,US,BROWARD,2019-11-16 17:00:00+00:00,26.054047,-80.257608
2019-11-16 12:00:00,Near Road,co,0.15,ppm,US,BROWARD,2019-11-16 17:00:00+00:00,26.131256,-80.167847


In [29]:
co_groupby = us_df_co.groupby('location')
co_first = co_groupby.first()

comean = us_df_co.groupby('location').mean()
comean = comean.rename(columns = {"value": "avgvalue", "coordinates.latitude": "lat", "coordinates.longitude": "lng"})

co = pd.merge(co_first, comean, on = 'location')
co = co.drop(['value','lat','lng'], axis=1)
co = co.rename(columns = {"coordinates.latitude": "coordinates_latitude", "coordinates.longitude": "coordinates_longitude"})

engine.execute("""CREATE TABLE co (
        location TEXT,
        parameter TEXT,
        unit TEXT,
        country TEXT,
        city TEXT,
        "date.utc" TIMESTAMP,
        "coordinates_latitude" FLOAT,
        "coordinates_longitude" FLOAT,
        avgvalue FLOAT
, primary key("location"));""")
co.to_sql(name='co', con=engine, index=True, index_label='location',if_exists='append')

co.head()

,parameter,unit,country,city,date.utc,coordinates_latitude,coordinates_longitude,avgvalue
location,,,,,,,,
77th and Dodge,co,ppm,US,Omaha-Council Bluffs,2019-11-16 16:00:00+00:00,41.259478,-96.034714,0.356066
ALLEN PARK,co,ppm,US,Detroit-Warren-Livonia,2019-11-16 16:00:00+00:00,42.228300,-83.209200,0.000159
AURORA HILLS,co,ppm,US,Washington-Arlington-Alexandria,2019-11-16 16:00:00+00:00,38.859402,-77.058899,0.490164
Anaheim,co,ppm,US,Los Angeles-Long Beach-Santa Ana,2019-11-16 16:00:00+00:00,33.830586,-117.938509,0.456393
Anaheim Near Road,co,ppm,US,ORANGE,2019-11-16 16:00:00+00:00,33.819305,-117.918759,0.725806


In [53]:
# api = openaq.OpenAQ()
# res2 = api.locations(coordinates='38.63, -90.20', radius='50000', limit=10000, df=True)

# # res2 = res2.query("value >= 0.0")
# # g = sns.FacetGrid(data=res, col='location', col_wrap=4, size=3, sharex=True, sharey=False)

# # g.map(plt.hist, "value")
# # print(res)
# res.dtypes

# stl_df2 = pd.DataFrame(res2)
# stl_df2